In [1]:
!git clone https://github.com/mateoKutnjak/pose_est.git

Cloning into 'pose_est'...


remote: Enumerating objects: 21, done.


remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.


remote: Total 21 (delta 6), reused 10 (delta 0), pack-reused 0


Unpacking objects: 100% (21/21), done.


In [ ]:
import sys
from google.colab import drive

import os
import pprint

from pose_est.dataset import MPII_dataset
from pose_est.models import HourglassModel
import tensorflow as tf
import keras.backend as k
import argparse

In [ ]:
sys.path.append('/content/pose_est')
drive.mount('/content/drive')

In [ ]:
if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.Session(tpu_address) as session:
        devices = session.list_devices()
    
    print('TPU devices:')
    pprint.pprint(devices)

In [ ]:
IMAGES_DIR = '/content/drive/My Drive/images'
ANNOTATIONS_JSON_FILE = '/content/drive/My Drive/annotations/annotations.json'
INPUT_SHAPE = (256, 256, 3)
OUTPUT_SHAPE = (64, 64, 16)
BATCH_SIZE = 8 * 64
STACKS_NUM = 4

In [ ]:
mpii_dataset = MPII_dataset(
        images_dir=IMAGES_DIR,
        annots_json_filename=ANNOTATIONS_JSON_FILE,
        input_shape=INPUT_SHAPE,
        output_shape=OUTPUT_SHAPE,
        type='train'
    )

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0

k.tensorflow_backend.set_session(tf.Session(config=config))

In [ ]:
hg_model = HourglassModel(
    dataset=mpii_dataset,
    batch_size=BATCH_SIZE,
    stacks_num=STACKS_NUM,
)

In [ ]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

hg_model = tf.contrib.tpu.keras_to_tpu_model(
    hg_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

In [ ]:
hg_model.build()

In [ ]:
hg_model.train()